In [86]:
library(R.matlab)
library(wavelets)
library(tuneR)
library(seewave)
library(data.table)
library(constellation)
library(GENEAread)

Loading required package: bitops

Loading required package: mmap



In [ ]:
baseName <- "frame"
wavesName = c("8PSK", "16QAM", "64QAM", "BPSK", "CPFSK", "GFSK", "QPSK", "PAM4")
dffile <- data.frame()

for(i in 1:1000) {
    for (w in wavesName) {
        n <- formatC(i, width=3, flag="0")
        name <- paste(baseName, w, n, ".mat", sep="")
        pathfile <- paste("../dataset/origin/", name, sep="")
        signal <- readMat(pathfile)
        values <- signal['frame']$frame
        
        #Time Domain
        t=seq(0,1023)
        s = Mod(values) * cos((pi*2.5*t) + Arg(values))
        
        #Frequency Domain (Fourier)
        f_s <- fft(s)
        
        #Frequency Domain (Wavelets)
        s_wavelets <- dwt(s, filter="la8", n.levels=3, boundary="periodic", fast=TRUE)
        s_ww <- c(s_wavelets@W$W1, s_wavelets@W$W2, s_wavelets@W$W3, c(1:128))
        
        #Zero-Cossing
        s_wave <- Wave(left=s, samp.rate=2500000000, bit=8)
        s_zc <- zc(s_wave, plot=FALSE)
        s_zc_x <- s_zc[,1]
        s_zc_y <- s_zc[,2]
        
        #autocorrelation
        ac <- acf(s, lag = 1023, plot=FALSE)
        
        #Constellation Shape Features
        tt <- seq(from = as.POSIXct("00:00:00", format="%H:%M:%S", tz="UTC"), length.out = 1024, by = "mins")
        DF = data.table( ID = 1, TIME = tt, VALUE = s)
        DT_C_SHAPE <- value_change(DF, value = 1.5, direction = "up", window_hours = 18, join_key = "ID", time_var = "TIME", value_var = "VALUE.V1", mult = "all")
        cs_current <- DT_C_SHAPE$CURRENT_VALUE[c(1:1024)]
        cs_prior <- DT_C_SHAPE$PRIOR_VALUE[c(1:1024)]
        
        #S Transform
        s_s <- stft(s, plot = FALSE, reassign = TRUE, win = 100)
        s_sa <- c(as.vector(s_s[[1]]), c(1:74))
        
        dfrow <- data.frame(file = name, wave = w, bandpass=values, timeDomain=s,
                            fourierDomain=f_s, waveletsDomain=s_ww, autoCorrelation=ac$acf,
                            zeroCrossX=s_zc_x, zeroCrossY=s_zc_y, sDomain=s_sa,
                            constellationCurrent=cs_current, constellationPrior=cs_prior)
        dffile <- rbind(dffile, dfrow)
    }
    
    #to facilitate the validation process, we created 10 different datasets
    if (i %% 100 == 0){
        nameBaseCSV = "data"
        pathCSV = "../dataset/transform/"
        nameCSV <- paste(pathCSV, nameBaseCSV, i, ".csv", sep="")
        write.csv(dffile, nameCSV, row.names = TRUE)
        dffile <- data.frame()
    }
}

